# Setting up the endpoint

In [1]:
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import csv_serializer

role = sagemaker.session.get_execution_role()

model_data='s3://inf1-compiled-bert-model/model.tar.gz'

pytorch_model = PyTorchModel(model_data=model_data, 
                             role=role,
                             entry_point='inference.py',
                             image='249828997522.dkr.ecr.us-east-1.amazonaws.com/bert-inf1-serving:latest',
                             framework_version='1.5.0',
                             enable_cloudwatch_metrics=True)

predictor = pytorch_model.deploy(instance_type='ml.inf1.xlarge', initial_instance_count=1)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Your model is not compiled. Please compile your model before using Inferentia.


-------------!

In [4]:
import torch
import transformers
from transformers import BertTokenizer
from transformers import BertModel
import math
import numpy as np
import io
from sagemaker.predictor import csv_serializer, json_deserializer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pickle 

def numpy_bytes_serializer(data):
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

# vnd indicates vendor-specific MIME types, which means they are MIME types that were 
# introduced by corporate bodies rather than e.g. an Internet consortium.


#-predictor.serializer = numpy_bytes_serializer
#-predictor.deserializer = csv_serializer

predictor.content_type = 'application/binary'
predictor.serializer = None
predictor.deserializer = None

sentence1="If you set your goals ridiculously high and it's a failure, you will fail above everyone else's success."
sentence2="The greatest glory in living lies not in never falling, but in rising every time we fall."
sentence3="If you set your goals ridiculously high and it's a failure, you will fail above everyone else's success. If you set your goals ridiculously high and it's a failure, you will fail above everyone else's success. If you set your goals ridiculously high and it's a failure, you will fail above everyone else's success."

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

cos = torch.nn.CosineSimilarity()

encoded_sentence = tokenizer.encode_plus(sentence1, sentence3, max_length=128, pad_to_max_length=True, return_tensors="pt", truncation=True)
encoded_sentence_tuple = encoded_sentence['input_ids'], encoded_sentence['attention_mask'], encoded_sentence['token_type_ids'] 

try:
    pickled_bytes = pickle.dumps(encoded_sentence_tuple)
    raw_bytes = predictor.predict(pickled_bytes)
except:
    pass
    
    
#print(pickle.loads(raw_bytes))

In [5]:
import random

s_nouns = ["A dude", "My mom", "The king", "Some guy", "A cat with rabies", "A sloth", "Your homie", "This cool guy my gardener met yesterday", "Superman"]
p_nouns = ["These dudes", "Both of my moms", "All the kings of the world", "Some guys", "All of a cattery's cats", "The multitude of sloths living under your bed", "Your homies", "Like, these, like, all these people", "Supermen"]
s_verbs = ["eats", "kicks", "gives", "treats", "meets with", "creates", "hacks", "configures", "spies on", "retards", "meows on", "flees from", "tries to automate", "explodes"]
p_verbs = ["eat", "kick", "give", "treat", "meet with", "create", "hack", "configure", "spy on", "retard", "meow on", "flee from", "try to automate", "explode"]
infinitives = ["to make a pie.", "for no apparent reason.", "because the sky is green.", "for a disease.", "to be able to make toast explode.", "to know more about archeology."]


In [6]:
import numpy as np 
import datetime
import math
import time
import boto3   
import matplotlib.pyplot as plt


endpoint_name=predictor.endpoint
total_runs=1000

In [8]:
print('Running {} inferences for {}:'.format(total_runs, endpoint_name))

client_times = []
errors_list = []
cw_start = datetime.datetime.utcnow()

errors = 0

for i in range(total_runs):    
    
    client_start = time.time()
    
    sentence_random = (random.choice(s_nouns) + ' ' + random.choice(s_verbs) + ' ' + random.choice(s_nouns).lower() or random.choice(p_nouns).lower() + ' ' + random.choice(infinitives))
    encoded_sentence = tokenizer.encode_plus(sentence1, sentence_random, max_length=128, pad_to_max_length=True, return_tensors="pt", truncation=True)
    encoded_sentence_tuple = encoded_sentence['input_ids'], encoded_sentence['attention_mask'], encoded_sentence['token_type_ids'] 
    
    pickled_bytes = pickle.dumps(encoded_sentence_tuple)

    try:
        raw_bytes = predictor.predict(pickled_bytes)
        errors_list.append(20)
    except:
        errors += 1
        errors_list.append(30)
        pass
    
    client_end = time.time()
    client_times.append((client_end - client_start)*1000)
    
print('\nErrors - {:.4f} out of {:.4f} total runs | {:.4f}% \n'.format(errors, total_runs, (errors/total_runs)*100))
errors = 0
    
    
cw_end = datetime.datetime.utcnow()    
    
print('Client end-to-end latency percentiles:')
client_avg = np.mean(client_times)
client_p50 = np.percentile(client_times, 50)
client_p90 = np.percentile(client_times, 90)
client_p95 = np.percentile(client_times, 95)
client_p100 = np.percentile(client_times, 100)
print('Avg | P50 | P90 | P95 | P100')
print('{:.4f} | {:.4f} | {:.4f} | {:.4f}\n'.format(client_avg, client_p50, client_p90, client_p95, client_p100))

print('Getting Cloudwatch:')
cloudwatch = boto3.client('cloudwatch')
statistics=['SampleCount', 'Average', 'Minimum', 'Maximum']
extended=['p50', 'p90', 'p95', 'p100']

# Give 5 minute buffer to end
cw_end += datetime.timedelta(minutes=5)

# Period must be 1, 5, 10, 30, or multiple of 60
# Calculate closest multiple of 60 to the total elapsed time
factor = math.ceil((cw_end - cw_start).total_seconds() / 60)
period = factor * 60
print('Time elapsed: {} seconds'.format((cw_end - cw_start).total_seconds()))
print('Using period of {} seconds\n'.format(period))

cloudwatch_ready = False
# Keep polling CloudWatch metrics until datapoints are available
while not cloudwatch_ready:
  time.sleep(30)
  print('Waiting 30 seconds ...')
  # Must use default units of microseconds
  model_latency_metrics = cloudwatch.get_metric_statistics(MetricName='ModelLatency',
                                             Dimensions=[{'Name': 'EndpointName',
                                                          'Value': endpoint_name},
                                                         {'Name': 'VariantName',
                                                          'Value': "AllTraffic"}],
                                             Namespace="AWS/SageMaker",
                                             StartTime=cw_start,
                                             EndTime=cw_end,
                                             Period=period,
                                             Statistics=statistics,
                                             ExtendedStatistics=extended
                                             )
  # Should be 1000
  if len(model_latency_metrics['Datapoints']) > 0:
    print('{} latency datapoints ready'.format(model_latency_metrics['Datapoints'][0]['SampleCount']))
    side_avg = model_latency_metrics['Datapoints'][0]['Average'] / total_runs
    side_p50 = model_latency_metrics['Datapoints'][0]['ExtendedStatistics']['p50'] / total_runs
    side_p90 = model_latency_metrics['Datapoints'][0]['ExtendedStatistics']['p90'] / total_runs
    side_p95 = model_latency_metrics['Datapoints'][0]['ExtendedStatistics']['p95'] / total_runs
    side_p100 = model_latency_metrics['Datapoints'][0]['ExtendedStatistics']['p100'] / total_runs
    print('Avg | P50 | P90 | P95 | P100')
    print('{:.4f} | {:.4f} | {:.4f} | {:.4f}\n'.format(side_avg, side_p50, side_p90, side_p95, side_p100))

    cloudwatch_ready = True
    
    #embeddings_returned = pickle.loads(raw_bytes)
    #s1 = embeddings_returned[1]
    #print(pickle.loads(raw_bytes))

Running 1000 inferences for bert-inf1-serving-2020-09-17-07-52-59-567:

Errors - 0.0000 out of 1000.0000 total runs | 0.0000% 

Client end-to-end latency percentiles:
Avg | P50 | P90 | P95 | P100
27.5222 | 26.8953 | 29.5256 | 31.0617

Getting Cloudwatch:
Time elapsed: 327.524155 seconds
Using period of 360 seconds

Waiting 30 seconds ...
206.0 latency datapoints ready
Avg | P50 | P90 | P95 | P100
19.5647 | 19.4170 | 21.4780 | 22.5970

